# Architecture A — single-agent baseline

Questa notebook prepara l'ambiente su Colab, clona la repo, configura i modelli HuggingFace e lancia il grafo per l'architettura **A** (single-agent → tester). Salva i log in `log/`.

Note operative:
- Imposta un token HuggingFace valido (HF Inference API) quando richiesto.
- Lancia pochi task della APPS per evitare costi/tempo eccessivi.
- I log sono salvati sia su stdout sia in file JSONL/LOG per analisi successive.

In [ ]:
import os
import sys
import subprocess
import pathlib

REPO_URL = "https://github.com/LLM4SE-group-15/ArchitecturesForCodeDevelopmentWithLLMs.git"
REPO_DIR = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

os.chdir(REPO_DIR)
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "pip"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)

print(f"Using repo at {REPO_DIR.resolve()}")

Using repo at /content/ArchitecturesForCodeDevelopmentWithLLMs


In [7]:
!pip install -r requirements.txt

In [21]:
import os
import getpass

from huggingface_hub import login

# Always provide the token explicitly:
# - avoids Colab "Secrets" timeouts when running via VS Code extension
# - ensures gated models (if used) can be accessed

if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter HF_TOKEN (kept hidden): ")

login(token=os.environ["HF_TOKEN"], add_to_git_credential=False)

os.environ["ARCHITECTURE"] = "A"
print("ARCHITECTURE set to", os.environ["ARCHITECTURE"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


ARCHITECTURE set to A


In [22]:
from huggingface_hub import HfApi

api = HfApi()
try:
    user_info = api.whoami(token=os.environ["HF_TOKEN"])
    print("Logged in to Hugging Face as:", user_info.get("name") or user_info.get("user"))
except Exception as exc:
    print("Login check failed:", exc)

Logged in to Hugging Face as: emanueleR3


In [23]:
import json
import time
import logging
import pathlib
import os

from datetime import datetime

DEFAULT_ROOT = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")
ROOT = DEFAULT_ROOT if DEFAULT_ROOT.exists() else pathlib.Path.cwd()

sys.path.insert(0, str(ROOT))

LOG_DIR = ROOT / "log"
LOG_DIR.mkdir(exist_ok=True)

logger = logging.getLogger("architecture_A")
logger.setLevel(logging.INFO)
if logger.handlers:
    logger.handlers.clear()
formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
file_handler = logging.FileHandler(LOG_DIR / "architecture_A.log")
stream_handler = logging.StreamHandler()
for handler in (file_handler, stream_handler):
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Logger ready. Repo root: %s", ROOT)
logger.info("Log files: %s", LOG_DIR)
print("Logs ->", LOG_DIR)

2026-01-03 10:35:28,989 | INFO | Logger ready. Repo root: /content/ArchitecturesForCodeDevelopmentWithLLMs
INFO:architecture_A:Logger ready. Repo root: /content/ArchitecturesForCodeDevelopmentWithLLMs
2026-01-03 10:35:28,991 | INFO | Log files: /content/ArchitecturesForCodeDevelopmentWithLLMs/log
INFO:architecture_A:Log files: /content/ArchitecturesForCodeDevelopmentWithLLMs/log


Logs -> /content/ArchitecturesForCodeDevelopmentWithLLMs/log


In [43]:
import time
from src.data.task_loader import APPSTaskLoader
from src.graph.graph import run_graph
from src.agents.llm import Architecture

ARCH = Architecture.A

def run_sample_tasks(
    per_level: int = 10,
    split: str = "test",
    difficulties: tuple[str, ...] = ("introductory",),
    shuffle: bool = True,
):
    loader = APPSTaskLoader(split=split)
    if shuffle:
        loader._dataset = loader.dataset.shuffle(seed=int(time.time()))

    tasks = []
    for diff in difficulties:
        tasks.extend(loader.load_by_difficulty(diff, limit=per_level))

    results = []
    total = len(tasks)
    logger.info("Loaded %s tasks (%s per difficulty: %s)", total, per_level, ", ".join(difficulties))

    for idx, task in enumerate(tasks, 1):
        logger.info("Running %s/%s %s (%s)", idx, total, task.task_id, task.difficulty)
        start = time.time()
        state = run_graph(
            task_id=task.task_id,
            task_description=task.question,
            test_inputs=task.inputs,
            test_outputs=task.outputs,
            architecture=ARCH,
        )

        print("State:", state)
        print()

        elapsed = time.time() - start
        record = {
            "task_id": task.task_id,
            "difficulty": task.difficulty,
            "architecture": str(ARCH.value),
            "test_passed": state["test_passed"],
            "developer_tier": state.get("developer_tier"),
            "escalations": state["escalations"],
            "story_points_initial": state.get("story_points_initial"),
            "story_points_final": state.get("story_points_current"),
            "elapsed_seconds": elapsed,
        }
        results.append(record)
        logger.info(
            "Finished %s | pass=%s tier=%s escalations=%s elapsed=%.1fs",
            task.task_id,
            state["test_passed"],
            record["developer_tier"],
            record["escalations"],
            elapsed,
        )
        with open(LOG_DIR / "architecture_A.jsonl", "a", encoding="utf-8") as f:
            f.write(json.dumps(record) + "\n")
    return results

sample_results = run_sample_tasks(per_level=30, difficulties=("introductory",), shuffle=True)
sample_results

Loading APPS dataset (test split)...


2026-01-03 11:11:09,876 | INFO | Loaded 30 tasks (30 per difficulty: introductory)
INFO:architecture_A:Loaded 30 tasks (30 per difficulty: introductory)
2026-01-03 11:11:09,878 | INFO | Running 1/30 apps_4956 (introductory)
INFO:architecture_A:Running 1/30 apps_4956 (introductory)


Loaded 5000 tasks.


2026-01-03 11:11:13,546 | INFO | Finished apps_4956 | pass=True tier=None escalations=0 elapsed=3.7s
INFO:architecture_A:Finished apps_4956 | pass=True tier=None escalations=0 elapsed=3.7s
2026-01-03 11:11:13,549 | INFO | Running 2/30 apps_4198 (introductory)
INFO:architecture_A:Running 2/30 apps_4198 (introductory)


State: {'task_id': 'apps_4956', 'task_description': 'Anders talks in the Swedish dialect of östgötska. Unfortunately, this makes it somewhat hard to get a programming job in the Swedish capital of Stockholm. The trendy Stockholm hipsters only accept applicants speaking the standard Swedish dialect, rikssvenska.\n\nTo increase his chances of passing interviews, he wishes to practice talking rikssvenska. To help him with this, he wants you to write a program that can determine whenever he accidentally reverts to speaking östgötska.\n\nA simple way of determining if a sentence is written in östgötska is if at least $40\\% $ of the words in the sentence contains the letter ä. For simplicity, this is encoded as the letter combination ae (meaning any appearance of the substring ae is to be regarded as an occurrence of the letter ä).\n\n-----Input-----\nThe first and only line of input contains a sequence of space-separated words. Each word consists only of letters a-z. There are at most $100

...

2026-01-03 11:14:03,661 | INFO | Finished apps_4900 | pass=False tier=None escalations=0 elapsed=3.8s
INFO:architecture_A:Finished apps_4900 | pass=False tier=None escalations=0 elapsed=3.8s
2026-01-03 11:14:03,663 | INFO | Running 30/30 apps_4134 (introductory)
INFO:architecture_A:Running 30/30 apps_4134 (introductory)


State: {'task_id': 'apps_4900', 'task_description': 'Somewhere in a mountainous region of the world is a collection of $n$ villages. Connecting these villages to one another is a series of roads, always running directly between two villages and allowing travel in both directions. Due to the treacherous landscape, building these roads is expensive, so the minimum number of roads have been constructed such that each village can reach every other village via a sequence of roads. \n\nTrade between these villages is very important, since not every village has access to the same supply of natural resources. Many villages produce the same resource, however, so it is useful for villages to know their relative distance to other villages so that they can choose trading partners to minimize overall trading costs. Note that the distance between two villages $a$ and $b$ is the sum of the lengths of the individual roads on the shortest path that connects $a$ and $b$.\n\nA project has been underway t

2026-01-03 11:14:10,022 | INFO | Finished apps_4134 | pass=False tier=None escalations=0 elapsed=6.4s
INFO:architecture_A:Finished apps_4134 | pass=False tier=None escalations=0 elapsed=6.4s


State: {'task_id': 'apps_4134', 'task_description': 'There is a rectangular grid of size $n \\times m$. Each cell has a number written on it; the number on the cell ($i, j$) is $a_{i, j}$. Your task is to calculate the number of paths from the upper-left cell ($1, 1$) to the bottom-right cell ($n, m$) meeting the following constraints:\n\n  You can move to the right or to the bottom only. Formally, from the cell ($i, j$) you may move to the cell ($i, j + 1$) or to the cell ($i + 1, j$). The target cell can\'t be outside of the grid.  The xor of all the numbers on the path from the cell ($1, 1$) to the cell ($n, m$) must be equal to $k$ (xor operation is the bitwise exclusive OR, it is represented as \'^\' in Java or C++ and "xor" in Pascal). \n\nFind the number of such paths in the given grid.\n\n\n-----Input-----\n\nThe first line of the input contains three integers $n$, $m$ and $k$ ($1 \\le n, m \\le 20$, $0 \\le k \\le 10^{18}$) — the height and the width of the grid, and the numbe

[{'task_id': 'apps_4956',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': True,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 3.667419195175171},
 {'task_id': 'apps_4198',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 4.9719398021698},
 {'task_id': 'apps_4409',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 4.81211256980896},
 {'task_id': 'apps_4646',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 7.549254

In [39]:
!cd log && cat architecture_A.jsonl

{"task_id": "apps_4000", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 21.43400001525879}
{"task_id": "apps_4000", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 24.04499888420105}
{"task_id": "apps_4406", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 2.9327504634857178}
{"task_id": "apps_4326", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 2.3970892429351807}
{"task_id": "apps_4118", "difficulty": "introductory",